<a href="https://colab.research.google.com/github/muhammadali74/MLAlgorithms/blob/master/Assignment_1_sn07590.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 335: Introduction to Large Language Models
## Assignment 01
### **Total Marks**: 100
### **Deadline**: Sunday, 3rd March, 2024, 11:59 PM
### **Name**: Syed Muhammad Ali Naqvi
### **ID**: sn07590

#Instructions

1. Please rename your notebook as *Assignment_1_aa1234.ipynb* before the final submission. Notebooks which do not follow appropriate naming convention will not be graded.

2. Please submit your own work. If you have any questions, please feel free to reach out to the course instructors or RA.



# Assignment Overview

In this assignment, you are required to fine tune a LLM model of your that classifies which human value category a textual arguement belongs to. Your model will evaluated against 1-baseline, random-baseline results on the following dataset: test, Nahjalbalagha, Zhihu


# Setup



In [1]:
# IMPORT ALL YOUR LIBRARIES
# SUGGESTED LIBRARIES
!pip install accelerate -U

import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, AutoModelForSequenceClassification, TrainingArguments, EvalPrediction
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.4 MB/s eta 0:00:00


# Download Files


##Evaluator

In [2]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/evaluator/evaluator.py

--2024-03-02 03:07:21--  https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/evaluator/evaluator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8708 (8.5K) [text/plain]
Saving to: ‘evaluator.py’

evaluator.py        100%[===================>]   8.50K  --.-KB/s    in 0s      

2024-03-02 03:07:22 (60.1 MB/s) - ‘evaluator.py’ saved [8708/8708]



## 1-Baseline

In [3]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/1-baseline/1-baseline.py

--2024-03-02 03:07:22--  https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/1-baseline/1-baseline.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3613 (3.5K) [text/plain]
Saving to: ‘1-baseline.py’

1-baseline.py       100%[===================>]   3.53K  --.-KB/s    in 0s      

2024-03-02 03:07:22 (37.5 MB/s) - ‘1-baseline.py’ saved [3613/3613]



## Random-Baseline

In [4]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/random-baseline/random-baseline.py

--2024-03-02 03:07:22--  https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/random-baseline/random-baseline.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5442 (5.3K) [text/plain]
Saving to: ‘random-baseline.py’

random-baseline.py  100%[===================>]   5.31K  --.-KB/s    in 0s      

2024-03-02 03:07:23 (63.8 MB/s) - ‘random-baseline.py’ saved [5442/5442]



## Dataset Files

In [5]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://zenodo.org/api/records/10564870/files-archive

--2024-03-02 03:07:23--  https://zenodo.org/api/records/10564870/files-archive
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.103.159, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘files-archive’

files-archive           [              <=>   ]  54.70M  15.6MB/s    in 4.3s    

2024-03-02 03:07:29 (12.6 MB/s) - ‘files-archive’ saved [57353430]



In [6]:
# DO NOT EDIT
# RUN ONLY ONCE
!unzip /content/files-archive -d Dataset

Archive:  /content/files-archive
 extracting: Dataset/value-categories.json  
 extracting: Dataset/arguments-validation-zhihu.tsv  
 extracting: Dataset/level1-labels-training.tsv  
 extracting: Dataset/arguments-test-nahjalbalagha.tsv  
 extracting: Dataset/arguments-test.tsv  
 extracting: Dataset/arguments-training.tsv  
 extracting: Dataset/arguments-validation.tsv  
 extracting: Dataset/labels-test-nahjalbalagha.tsv  
 extracting: Dataset/labels-test-nyt.tsv  
 extracting: Dataset/labels-test.tsv  
 extracting: Dataset/labels-training.tsv  
 extracting: Dataset/level1-labels-test-nahjalbalagha.tsv  
 extracting: Dataset/level1-labels-test.tsv  
 extracting: Dataset/level1-labels-test-nyt.tsv  
 extracting: Dataset/labels-validation.tsv  
 extracting: Dataset/labels-validation-zhihu.tsv  
 extracting: Dataset/meta-arguments-g.tsv  
 extracting: Dataset/meta-arguments-c.tsv  
 extracting: Dataset/meta-arguments-e.tsv  
 extracting: Dataset/meta-arguments-f.tsv  
 extracting: Dataset

In [7]:
!mkdir /content/Dataset/zhihu
!mkdir /content/Dataset/nahjalbalagha
!mkdir /content/Dataset/train
!mkdir /content/Dataset/test
!mkdir /content/Dataset/validation

In [8]:
!mv /content/Dataset/*-zhihu.tsv /content/Dataset/zhihu
!mv /content/Dataset/*-nahjalbalagha.tsv /content/Dataset/nahjalbalagha
!mv /content/Dataset/*-nahjalbalagha.tsv /content/Dataset/nahjalbalagha
!mv /content/Dataset/*-training.tsv /content/Dataset/train
!mv /content/Dataset/*-test.tsv /content/Dataset/test
!mv /content/Dataset/*-validation.tsv /content/Dataset/validation

mv: cannot stat '/content/Dataset/*-nahjalbalagha.tsv': No such file or directory


# Background Information

## Human Value Detection 2023 <br/>
## SemEval 2023 Task 4. ValueEval: Identification of Human Values behind Arguments



Given a textual argument and a human value category, classify whether or not the argument draws on that category. This task uses a set of 20 value categories compiled from the social science literature and described in our [ACL paper](https://webis.de/publications.html#kiesel_2022b). Arguments are given as premise text, conclusion text, and binary stance of the premise to the conclusion ("in favor of" or "against").

The 20 value categories are shown here on Schwartz' value continuum below:

[![JEPBxUu.md.png](https://iili.io/JEPBxUu.md.png)](https://freeimage.host/i/JEPBxUu)







## Data


Data is provided as tab-separated values files with one header line. The arguments-validation.tsv files contain one argument per line: its unique argument ID, the conclusion, the premise's stance towards the conclusion, and the premise itself. Example with tab-separated columns are shown below

<pre><span class="column">Argument ID</span>	<span class="column">Conclusion</span>	<span class="column">Stance</span>	<span class="column">Premise</span>
<span class="column">A01010</span>	<span class="column">We should prohibit school prayer</span>	<span class="column">against</span>	<span class="column">it should be allowed if the student wants to pray as long as it is not interfering with his classes</span>
<span class="column">A01011</span>	<span class="column">We should abolish the three-strikes laws</span>	<span class="column">in favor of</span>	<span class="column">three strike laws can cause young people to be put away for life without a chance to straight out their life</span>
<span class="column">A01012</span>	<span class="column">The use of public defenders should be mandatory</span>	<span class="column">in favor of</span>	<span class="column">the use of public defenders should be mandatory because some people don't have money for a lawyer and this would help those that don't</span>
</pre>

The labels-validation.tsv  files also contain one argument per line: its unique argument ID and one column for each of the 20 value categories with a 1 meaning that the argument resorts to the value category and a 0 that not. Example with tab-separated columns are shown below:

<pre><span class="column">Argument ID</span>	<span class="column">Self-direction: thought</span>	<span class="column">Self-direction: action</span>	<span class="column">Stimulation</span>	<span class="column">Hedonism</span>	<span class="column">Achievement</span>	<span class="column">Power: dominance</span>	<span class="column">Power: resources</span>	<span class="column">Face</span>	<span class="column">Security: personal</span>	<span class="column">Security: societal</span>	<span class="column">Tradition</span>	<span class="column">Conformity: rules</span>	<span class="column">Conformity: interpersonal</span>	<span class="column">Humility</span>	<span class="column">Benevolence: caring</span>	<span class="column">Benevolence: dependability</span>	<span class="column">Universalism: concern</span>	<span class="column">Universalism: nature</span>	<span class="column">Universalism: tolerance</span>	<span class="column">Universalism: objectivity</span>
<span class="column">A01010</span>	<span class="column">1</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>
<span class="column">A01011</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">1</span>
<span class="column">A01012</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>
</pre>

In addition, there are other datasets for evaluating the robustness of our model: validation-zhihu from the recommendation and hotlist section of the Chinese question-answering website Zhihu, test-nahjalbalagha from and based on the Nahj al-Balagha.



## Evaluation

Runs are evaluated on the basis of F1-score, Precision, and Recall: averaged over all value categories and for each category individually.

## Baseline Results

In [9]:
# DO NOT EDIT
# RUN ONLY ONCE
!mkdir /content/baseline
!mkdir /content/output

### 1-Baseline

#### Test Dataset

In [10]:
# DO NOT EDIT
!python3 /content/1-baseline.py --inputDataset /content/Dataset/test --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/test/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/test/arguments-test.tsv
Labeling 1576 instances
Detected values: {'Benevolence: dependability', 'Self-direction: action', 'Universalism: objectivity', 'Power: resources', 'Conformity: rules', 'Universalism: tolerance', 'Security: societal', 'Security: personal', 'Humility', 'Benevolence: caring', 'Conformity: interpersonal', 'Hedonism', 'Achievement', 'Power: dominance', 'Universalism: concern', 'Tradition', 'Stimulation', 'Face', 'Universalism: nature', 'Self-direction: thought'}
Writing run file
Reading /content/Dataset/test/labels-test.tsv
Reading /content/baseline/run.tsv
Truth labels: 1576
Run labels:   1576
measure {
 key: "F1"
 value: "0.26293020308065357"
}
measure {
 key: "Precision"
 value: "0.15136421319796953"
}
measure {
 key: "Recall"
 value: "1.0"
}


#### Zhihu

In [11]:
# DO NOT EDIT
!python3 /content/1-baseline.py --inputDataset /content/Dataset/zhihu/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/zhihu/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/zhihu/arguments-validation-zhihu.tsv
Labeling 100 instances
Detected values: {'Hedonism', 'Self-direction: thought', 'Universalism: concern', 'Humility', 'Face', 'Conformity: interpersonal', 'Security: societal', 'Universalism: objectivity', 'Security: personal', 'Achievement', 'Power: resources', 'Tradition', 'Self-direction: action', 'Stimulation', 'Conformity: rules', 'Power: dominance', 'Universalism: tolerance', 'Universalism: nature', 'Benevolence: dependability', 'Benevolence: caring'}
Writing run file
Reading /content/Dataset/zhihu/labels-validation-zhihu.tsv
Reading /content/baseline/run.tsv
Truth labels: 100
Run labels:   100
measure {
 key: "F1"
 value: "0.2292179045745204"
}
measure {
 key: "Precision"
 value: "0.12944444444444445"
}
measure {
 key: "Recall"
 value: "1.0"
}


#### Nahjalbalagha

In [12]:
# DO NOT EDIT
!python3 /content/1-baseline.py --inputDataset /content/Dataset/nahjalbalagha/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/nahjalbalagha/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/nahjalbalagha/arguments-test-nahjalbalagha.tsv
Labeling 279 instances
Detected values: {'Humility', 'Hedonism', 'Security: societal', 'Universalism: objectivity', 'Conformity: interpersonal', 'Power: resources', 'Universalism: tolerance', 'Benevolence: caring', 'Power: dominance', 'Self-direction: action', 'Face', 'Security: personal', 'Universalism: nature', 'Tradition', 'Achievement', 'Self-direction: thought', 'Stimulation', 'Conformity: rules', 'Benevolence: dependability', 'Universalism: concern'}
Writing run file
Reading /content/Dataset/nahjalbalagha/labels-test-nahjalbalagha.tsv
Reading /content/baseline/run.tsv
Truth labels: 279
Run labels:   279
measure {
 key: "F1"
 value: "0.12845882944826426"
}
measure {
 key: "Precision"
 value: "0.0686379928315412"
}
measure {
 key: "Recall"
 value: "1.0"
}


### Random-Baseline


#### Test

In [13]:
# DO NOT EDIT
!python3 /content/random-baseline.py --inputDataset /content/Dataset/test --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/test/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/test/arguments-test.tsv
Labeling 1576 instances
Detected values: {'Achievement', 'Security: personal', 'Face', 'Self-direction: action', 'Benevolence: caring', 'Stimulation', 'Conformity: rules', 'Universalism: concern', 'Universalism: objectivity', 'Tradition', 'Power: resources', 'Conformity: interpersonal', 'Power: dominance', 'Benevolence: dependability', 'Security: societal', 'Hedonism', 'Universalism: nature', 'Humility', 'Self-direction: thought', 'Universalism: tolerance'}
Writing run file
Reading /content/Dataset/test/labels-test.tsv
Reading /content/baseline/run.tsv
Truth labels: 1576
Run labels:   1576
measure {
 key: "F1"
 value: "0.16608921808129248"
}
measure {
 key: "Precision"
 value: "0.15550348857211035"
}
measure {
 key: "Recall"
 value: "0.17822144775672288"
}


#### Zhihu


In [14]:
# DO NOT EDIT
!python3 /content/random-baseline.py --inputDataset /content/Dataset/zhihu/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/zhihu/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/zhihu/arguments-validation-zhihu.tsv
Labeling 100 instances
Detected values: {'Security: societal', 'Self-direction: action', 'Face', 'Stimulation', 'Security: personal', 'Benevolence: caring', 'Power: dominance', 'Power: resources', 'Conformity: interpersonal', 'Humility', 'Tradition', 'Self-direction: thought', 'Universalism: objectivity', 'Benevolence: dependability', 'Achievement', 'Conformity: rules', 'Universalism: concern', 'Hedonism', 'Universalism: tolerance', 'Universalism: nature'}
Writing run file
Reading /content/Dataset/zhihu/labels-validation-zhihu.tsv
Reading /content/baseline/run.tsv
Truth labels: 100
Run labels:   100
measure {
 key: "F1"
 value: "0.15573918118176303"
}
measure {
 key: "Precision"
 value: "0.12974292185194047"
}
measure {
 key: "Recall"
 value: "0.19476343548924194"
}


#### Nahjalbalagha

In [15]:
# DO NOT EDIT
!python3 /content/random-baseline.py --inputDataset /content/Dataset/nahjalbalagha/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/nahjalbalagha/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/nahjalbalagha/arguments-test-nahjalbalagha.tsv
Labeling 279 instances
Detected values: {'Power: dominance', 'Face', 'Universalism: concern', 'Security: societal', 'Benevolence: caring', 'Security: personal', 'Achievement', 'Universalism: objectivity', 'Benevolence: dependability', 'Self-direction: thought', 'Stimulation', 'Self-direction: action', 'Humility', 'Universalism: nature', 'Conformity: rules', 'Power: resources', 'Universalism: tolerance', 'Tradition', 'Conformity: interpersonal', 'Hedonism'}
Writing run file
Reading /content/Dataset/nahjalbalagha/labels-test-nahjalbalagha.tsv
Reading /content/baseline/run.tsv
Truth labels: 279
Run labels:   279
measure {
 key: "F1"
 value: "0.09227654825672356"
}
measure {
 key: "Precision"
 value: "0.06551438837109638"
}
measure {
 key: "Recall"
 value: "0.1560024250554273"
}


# Tasks

## [20 Points] Task 01 - Load Datasets

In this task, you are required to load the Training, Test, Validation, Nahjalbalagha & Zhihu into seperate dataframes.

In [16]:
# Write your code here
# Training Data
paths = {"training":"training", "test":"test", "validation":"validation", "zhihu":"validation-zhihu", "nahjalbalagha":"test-nahjalbalagha"}
datasets = {"training":{}, "test":{}, "validation":{}, "nahjalbalagha":{}, "zhihu":{}}
subset = ["arguments", "labels", "level1-labels"]


for i in datasets.keys():
  for j in subset:
    if i == "training":
      datasets[i][j] = (pd.read_csv(f"/content/Dataset/train/{j}-{i}.tsv", sep = "\t"))
    else:
      datasets[i][j] = (pd.read_csv(f"/content/Dataset/{i}/{j}-{paths[i]}.tsv", sep = "\t"))

print(datasets['training']['arguments'].shape)
print(datasets['training']['labels'].shape)

# datasets['training']['labels'].head()

(5393, 4)
(5393, 21)


## [10 Points] Task 02 - Define Tokenizer & Model


In this task, you are required to define the Tokenizer and LLM model of your choice.

In [ ]:
# Write your code here
# picking the best model from the leaderboard :)

from transformers import AutoTokenizer

model = "microsoft/phi-1_5"
model = "bert-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(model, do_lower_case = True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

rem_cols = [col for col in datasets['training']['labels'].columns[1:]]
id2label = {idx:label for idx, label in enumerate(rem_cols)}
label2id = {label:idx for idx, label in enumerate(rem_cols)}

LLM = AutoModelForSequenceClassification.from_pretrained(model, num_labels=20, problem_type="multi_label_classification", id2label=id2label, label2id=label2id)
print(LLM)




## [20 Points] Task 03 - Optimizer & Hyperparameters


In this task, you are required to define the hyperparameters & the optimizer for training your model.

In [62]:
# Write your code here

epochs = 4
# MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16

# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = len(train_df) * epochs)
training_args = TrainingArguments(".", evaluation_strategy="epoch",
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    num_train_epochs=epochs,
)
# training_args

## [20 Points] Task 04 -  Training Loop


In this task, you are required to implement the training loop for fine tuning your model. You are also required to plot on the same graph: Loss vs Epochs & Accuracy vs Epochs

In [63]:
# Tokeninzing inputs
import numpy as np

# print(rem_cols)
train_labels = datasets['training']['labels'][rem_cols]
train_labels = np.array(train_labels.values.tolist())
train_labels = train_labels.astype(np.float32)
train_args = datasets['training']['arguments']['Premise'].values.tolist()
train_args2 = datasets['training']['arguments']['Conclusion'].values.tolist()
# print(train_args)
# print(train_labels)
eval_labels = datasets['validation']['labels'][rem_cols]
eval_labels = np.array(eval_labels.values.tolist())
eval_labels = eval_labels.astype(np.float32).tolist()
eval_args = datasets['validation']['arguments']['Premise'].values.tolist()
eval_args2 = datasets['validation']['arguments']['Conclusion'].values.tolist()

train_encodings = tokenizer(train_args,train_args2, padding=True, truncation=True, max_length=512)
eval_encodings = tokenizer(eval_args, eval_args2, padding=True, truncation=True, max_length=512)
print(train_encodings[0].tokens)


class TextClassifierDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = TextClassifierDataset(train_encodings, train_labels)
eval_dataset = TextClassifierDataset(eval_encodings, eval_labels)

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [36]:
# Write your code here
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.2):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average = 'micro');
    recall = recall_score(y_true, y_pred, average = 'micro');
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy, 'precision': precision, 'recall': recall}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result


trainer = Trainer(model=LLM,args=training_args,train_dataset=train_dataset,eval_dataset=eval_dataset, compute_metrics=compute_metrics)

trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy,Precision,Recall
1,No log,0.354410,0.389772,0.623847,0.058544,0.665023,0.275672
2,0.367100,0.332308,0.480833,0.667408,0.081751,0.697459,0.366882
3,0.284800,0.328993,0.513069,0.687438,0.088608,0.667338,0.416732
4,0.284800,0.331414,0.514809,0.688584,0.082806,0.666001,0.419563


TrainOutput(global_step=1352, training_loss=0.3033712657951039, metrics={'train_runtime': 764.8564, 'train_samples_per_second': 28.204, 'train_steps_per_second': 1.768, 'total_flos': 1873770660823776.0, 'train_loss': 0.3033712657951039, 'epoch': 4.0})

## [10 Points]  Task 05 - Model Evaluation: Test Dataset

In this task, you are required your fine tuned model on the Test dataset using ``evaluator.py`` and compare your results with random and 1-baseline.

In [1]:
# evaluation function

def model_evaluator(dataset, output_dir, threshold = 0.2):
  prediction = []
  test_args = datasets[dataset]['arguments']['Premise'].values.tolist()
  test_args2 = datasets[dataset]['arguments']['Conclusion'].values.tolist()
  for idx, text in enumerate(test_args):
      text2 = test_args2[idx]
      inputs = tokenizer(text,text2, padding=True, truncation=True, return_tensors="pt")
      inputs = inputs.to(device)

      with torch.no_grad():
          outputs = LLM(**inputs)

      logits = outputs.logits
      probabilities = torch.sigmoid(logits)  # Apply sigmoid to convert logits to probabilities

      predictions.append(probabilities)


  predictions = np.array([tensor.cpu().numpy() for tensor in predictions])
  # Define the new threshold (e.g., 0.4)
  new_threshold = threshold

  label_cols = rem_cols
  # Create an empty DataFrame with the correct columns
  df_submission = pd.DataFrame(columns = label_cols)

  # Iterate through the rows of all_probabilities_array
  for row in predictions:
      # Create a mask for values greater than or equal to the new threshold
      mask = row >= new_threshold

      # Check if any value is above the threshold
      if mask.any():
          # If there is at least one value above the threshold, use the mask
          thresholded_row = mask.astype(int)
      else:
          # If there are no values above the threshold, take the top 1 value
          top_1_index = np.argmax(row)
          thresholded_row = np.zeros(len(label_cols), dtype=int)
          thresholded_row[top_1_index] = 1

      # Reshape the thresholded_row to match the length of the index
      thresholded_row = thresholded_row.reshape(1, -1)

      # Append the thresholded row to the DataFrame
      df_submission = df_submission.append(pd.DataFrame(thresholded_row, columns=label_cols), ignore_index=True)

  # Convert the DataFrame to integer type
  df_submission = df_submission.astype(int)

  ids = datasets[dataset]['labels']['Argument ID']
  run = pd.concat([ids, pred_df], axis = 1)
  run.to_csv(f'{output_dir}/run.tsv, sep='\t', index=False)



SyntaxError: unexpected character after line continuation character (<ipython-input-1-9074bcfbe150>, line 55)

In [49]:
#Write your code
!mkdir '/content/trained_test'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LLM.to(device)
# switchig model to evaluation mode


LLM.eval()
dataset = 'test'
output_dir = '/content/trained_test'
model_evaluator(dataset, output_dir)

# predictions = []
# # Process each text
# test_args = datasets['test']['arguments']['Premise'].values.tolist()
# test_args2 = datasets['test']['arguments']['Conclusion'].values.tolist()
# for idx, text in enumerate(test_args):
#     text2 = test_args2[idx]
#     inputs = tokenizer(text,text2, padding=True, truncation=True, return_tensors="pt")
#     inputs = inputs.to(device)

#     with torch.no_grad():
#         outputs = LLM(**inputs)

#     logits = outputs.logits
#     probabilities = torch.sigmoid(logits)  # Apply sigmoid to convert logits to probabilities

#     predictions.append(probabilities)


# predictions = np.array([tensor.cpu().numpy() for tensor in predictions])
# # Define the new threshold (e.g., 0.4)
# new_threshold = 0.5

# label_cols = datasets['training']['labels'].columns[1:]
# # Create an empty DataFrame with the correct columns
# df_submission = pd.DataFrame(columns = label_cols)

# # Iterate through the rows of all_probabilities_array
# for row in predictions:
#     # Create a mask for values greater than or equal to the new threshold
#     mask = row >= new_threshold

#     # Check if any value is above the threshold
#     if mask.any():
#         # If there is at least one value above the threshold, use the mask
#         thresholded_row = mask.astype(int)
#     else:
#         # If there are no values above the threshold, take the top 1 value
#         top_1_index = np.argmax(row)
#         thresholded_row = np.zeros(len(label_cols), dtype=int)
#         thresholded_row[top_1_index] = 1

#     # Reshape the thresholded_row to match the length of the index
#     thresholded_row = thresholded_row.reshape(1, -1)

#     # Append the thresholded row to the DataFrame
#     df_submission = df_submission.append(pd.DataFrame(thresholded_row, columns=label_cols), ignore_index=True)

# # Convert the DataFrame to integer type
# df_submission = df_submission.astype(int)

KeyboardInterrupt: 

In [58]:
# df_submission.head()

# ids = datasets['test']['labels']['Argument ID']
# run = pd.concat([ids, pred_df], axis = 1)
# run.head()
# run.to_csv('/content/trained/run.tsv', sep='\t', index=False)

mkdir: cannot create directory ‘/content/trained’: File exists


In [60]:
!python3 evaluator.py --inputDataset /content/Dataset/test/ --inputRun /content/trained_test --outputDataset /content
!head -n 12 /content/evaluation.prototext

Reading /content/Dataset/test/labels-test.tsv
Reading /content/trained/run.tsv
Truth labels: 1576
Run labels:   1576
measure {
 key: "F1"
 value: "0.39387456447527214"
}
measure {
 key: "Precision"
 value: "0.5546731244169848"
}
measure {
 key: "Recall"
 value: "0.3053532936131528"
}


## [10 Points]  Task 06 - Model Evaluation: Zhihu Dataset

In this task, you are required your fine tuned model on the Zhihu
 dataset using ``evaluator.py`` and compare your results with random and 1-baseline.

In [ ]:
dataset = 'zhihu'
output_dir = '/content/trained_zhihu'
model_evaluator(dataset, output_dir)

In [ ]:
!python3 evaluator.py --inputDataset /content/Dataset/zhihu/ --inputRun /content/trained_zhihu --outputDataset /content
!head -n 12 /content/evaluation.prototext

## [10 Points]  Task 07 - Model Evaluation: Nahjalbalagha Dataset

In this task, you are required your fine tuned model on the Nahjalbalagha dataset using ``evaluator.py`` and compare your results with random and 1-baseline.

In [ ]:
dataset = 'nahjalbalagha'
output_dir = '/content/trained_nahjalbalagha'
model_evaluator(dataset, output_dir)

In [ ]:
!python3 evaluator.py --inputDataset /content/Dataset/nahjalbalagha/ --inputRun /content/trained_nahjalbalagha --outputDataset /content
!head -n 12 /content/evaluation.prototext

# References

In this section, cite any resources or references that you use for solving this assignment.
